In [50]:
#!pip install visdom

In [3]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms

import torch.nn.init

In [4]:
import visdom
vis = visdom.Visdom(port = 6006)
vis.close(env="main")

Setting up a new session...


''

In [5]:
def loss_tracker(loss_plot, loss_value, num):
    vis.line(X = num, 
             Y = loss_value,
             win = loss_plot, update = 'append')

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [7]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [8]:
mnist_train = datasets.MNIST(root = 'data/', 
                             train = True, 
                             transform = transforms.ToTensor(), 
                             download = True)
mnist_test = datasets.MNIST(root = 'data/', 
                             train = False, 
                             transform = transforms.ToTensor(), 
                             download = True)

In [9]:
data_loader = torch.utils.data.DataLoader(dataset = mnist_train, 
                                          batch_size = batch_size,
                                         shuffle = True, 
                                         #drop_last = True
                                         )

In [10]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc1 = nn.Linear(3*3*128, 625)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(625, 10, bias = True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
    def forward(self, x):
        out = self.layer1(x)
        #print(out.shape)
        out = self.layer2(out)
        out = self.layer3(out)
        #print(out.shape)
        out = out.view(out.size(0), -1)
        #print(out.shape)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [11]:
model = CNN().to(device)
value = torch.Tensor(1,1,28,28).to(device)
print(model(value))

tensor([[ 1.1891e+23,  1.9616e+25, -2.3866e+25,  3.3678e+24, -1.0995e+25,
          3.9403e+24, -3.5095e+25,  1.2732e+23,  2.1851e+24,  3.3109e+25]],
       device='cuda:0', grad_fn=<AddmmBackward>)


In [12]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [13]:
loss_plt = vis.line(Y = torch.Tensor(1).zero_(), opts = dict(title='loss_tracker', legend=['loss'], showlegend=True))

In [ ]:
total_batch = len(data_loader)

for epoch in range(training_epochs):
    avg_cost = 0
    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad() # d이것을 뺴면 학습이 아예 안된다. 
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print('[Epoch: {}] cost = {}'.format(epoch + 1, avg_cost))
    loss_tracker(loss_plt, torch.Tensor([avg_cost]), torch.Tensor([epoch]))
print('Learning Finished!')

[Epoch: 1] cost = 0.16689211130142212
[Epoch: 2] cost = 0.04312305524945259
[Epoch: 3] cost = 0.030135832726955414
[Epoch: 4] cost = 0.022159185260534286
[Epoch: 5] cost = 0.017512165009975433


In [ ]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:' , accuracy.item())